In [1]:
import pandas as pd
import numpy as np
import requests as rq
from bs4 import BeautifulSoup as bs
import re
import os
from multiprocessing import Pool
from tqdm import tqdm
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options


In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time

options = Options()
options.add_argument('--disable-blink-features=AutomationControlled')
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
options.add_argument("--disable-extensions")

driver = webdriver.Chrome(options=options)
driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")

driver.get('https://www.sofascore.com/football/rankings/uefa')
time.sleep(1)  # Esperar a que cargue
ligas=[i['href'] for i in bs(driver.page_source, 'html.parser').find_all('a') if i['href'].startswith('/tournament/football') and 'world' not in i['href'] and 'europe' not in i['href']]
driver.quit()

In [3]:
ligas=list(set(ligas))  # Eliminar duplicados
ligas=['https://www.sofascore.com' + i for i in ligas]  # Añadir el dominio


In [4]:
ligas 

['https://www.sofascore.com/tournament/football/hungary/nb-i/187',
 'https://www.sofascore.com/tournament/football/ukraine/premier-league/218',
 'https://www.sofascore.com/tournament/football/austria/bundesliga/45',
 'https://www.sofascore.com/tournament/football/slovenia/prvaliga/212',
 'https://www.sofascore.com/tournament/football/cyprus/1-division/171',
 'https://www.sofascore.com/tournament/football/israel/israeli-premier-league/266',
 'https://www.sofascore.com/tournament/football/denmark/superliga/39',
 'https://www.sofascore.com/tournament/football/spain/laliga/8',
 'https://www.sofascore.com/tournament/football/romania/superliga/152',
 'https://www.sofascore.com/tournament/football/england/premier-league/17',
 'https://www.sofascore.com/tournament/football/serbia/mozzart-bet-superliga/210',
 'https://www.sofascore.com/tournament/football/scotland/premiership/36',
 'https://www.sofascore.com/tournament/football/netherlands/eredivisie/37',
 'https://www.sofascore.com/tournament/

In [5]:
# En lugar de usar requests, usa selenium para cada liga
def obtener_soup_liga(url):
    options = Options()
    options.add_argument('--headless')
    options.add_argument('--disable-blink-features=AutomationControlled')
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)
    
    driver = webdriver.Chrome(options=options)
    driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
    
    try:
        driver.get(url)
        time.sleep(2)  # Esperar a que cargue
        soup = bs(driver.page_source, 'html.parser')
        return soup
    finally:
        driver.quit()

# Probar con la primera liga
soup = obtener_soup_liga(ligas[0])

In [6]:
def get_equipos(soup):  # Cambiar parámetro a soup
    """
    Extrae equipos de un objeto soup de BeautifulSoup
    """
    try:
        equipos = soup.find_all('a')
        equipos = [i for i in equipos if i.get('href', '').startswith('/team/football') and 'world' not in i.get('href', '') and 'europe' not in i.get('href', '')]
        equipos = [i['href'] for i in equipos]
        equipos = list(set(equipos))  # Eliminar duplicados
        equipos = ['https://www.sofascore.com' + i for i in equipos]  # Añadir el dominio
        return equipos
    except Exception as e:
        print(f"Error al obtener equipos: {e}")
        return []


In [7]:
def obtener_todos_equipos_eficiente(ligas):
    """
    Versión más eficiente usando un solo driver
    """
    todos_los_equipos = []
    
    options = Options()
    options.add_argument('--headless')
    options.add_argument('--disable-blink-features=AutomationControlled')
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)
    
    driver = webdriver.Chrome(options=options)
    driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
    
    try:
        for i, liga_url in enumerate(ligas):
            print(f"Procesando liga {i+1}/{len(ligas)}: {liga_url}")
            
            try:
                driver.get(liga_url)
                time.sleep(2)
                
                soup = bs(driver.page_source, 'html.parser')
                equipos_liga = get_equipos(soup)
                
                print(f"  - Encontrados {len(equipos_liga)} equipos")
                todos_los_equipos.extend(equipos_liga)
                
                time.sleep(1)  # Pausa entre requests
                
            except Exception as e:
                print(f"  - Error en liga {liga_url}: {e}")
                continue
                
    finally:
        driver.quit()
    
    equipos_unicos = list(set(todos_los_equipos))
    print(f"\nTotal de equipos únicos encontrados: {len(equipos_unicos)}")
    
    return equipos_unicos

# Usar la versión eficiente
todos_equipos = obtener_todos_equipos_eficiente(ligas)

Procesando liga 1/28: https://www.sofascore.com/tournament/football/hungary/nb-i/187
  - Encontrados 12 equipos
Procesando liga 2/28: https://www.sofascore.com/tournament/football/ukraine/premier-league/218
  - Encontrados 16 equipos
Procesando liga 3/28: https://www.sofascore.com/tournament/football/austria/bundesliga/45
  - Encontrados 6 equipos
Procesando liga 4/28: https://www.sofascore.com/tournament/football/slovenia/prvaliga/212
  - Encontrados 2 equipos
Procesando liga 5/28: https://www.sofascore.com/tournament/football/cyprus/1-division/171
  - Encontrados 14 equipos
Procesando liga 6/28: https://www.sofascore.com/tournament/football/israel/israeli-premier-league/266
  - Encontrados 6 equipos
Procesando liga 7/28: https://www.sofascore.com/tournament/football/denmark/superliga/39
  - Encontrados 12 equipos
Procesando liga 8/28: https://www.sofascore.com/tournament/football/spain/laliga/8
  - Encontrados 20 equipos
Procesando liga 9/28: https://www.sofascore.com/tournament/foot

In [8]:
def obtener_soup_equipo(url):
    """
    Obtiene el soup de la página de un equipo específico.
    """
    options = Options()
    options.add_argument('--headless')
    options.add_argument('--disable-blink-features=AutomationControlled')
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)
    
    driver = webdriver.Chrome(options=options)
    driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")

    
    try:
        driver.get(url+'#tab:squad')
        wait = WebDriverWait(driver, 10)
        soup = bs(driver.page_source, 'html.parser')
        return soup
    finally:
        driver.quit()

In [9]:
def get_jugadores(team_url):
    """
    Obtiene los jugadores de un equipo dado su URL.
    """
    soup = obtener_soup_equipo(team_url)
    
    jugadores = []
    for jugador in soup.find_all('a'):
        if jugador['href'].startswith('/football/player') and 'world' not in jugador['href'] and 'europe' not in jugador['href']:
                jugadores.append('https://www.sofascore.com' + jugador['href'])
    
    return jugadores  # Eliminar duplicados
set(get_jugadores(todos_equipos[0]))

{'https://www.sofascore.com/football/player/anthony-losilla/119179',
 'https://www.sofascore.com/football/player/bamba-samuel/1083404',
 'https://www.sofascore.com/football/player/bernardo/828571',
 'https://www.sofascore.com/football/player/cristian-gamboa/117794',
 'https://www.sofascore.com/football/player/dani-de-wit/855854',
 'https://www.sofascore.com/football/player/erhan-masovic/833213',
 'https://www.sofascore.com/football/player/gerrit-holtmann/253517',
 'https://www.sofascore.com/football/player/giorgos-masouras/592560',
 'https://www.sofascore.com/football/player/ibrahima-sissoko/829093',
 'https://www.sofascore.com/football/player/ivan-ordets/137673',
 'https://www.sofascore.com/football/player/jakov-medic/872307',
 'https://www.sofascore.com/football/player/koji-miyoshi/358984',
 'https://www.sofascore.com/football/player/lennart-koerdt/1634278',
 'https://www.sofascore.com/football/player/mats-pannewig/1197157',
 'https://www.sofascore.com/football/player/matus-bero/1872

In [10]:
equipos_df = pd.DataFrame(todos_equipos, columns=['url'])
equipos_df.to_csv('equipos_sofascore.csv', index=False)

In [11]:
todos_equipos = pd.read_csv('equipos_sofascore.csv')['url'].tolist()

In [12]:
def obtener_todos_jugadores_secuencial(todos_equipos):
    """
    Obtiene todos los jugadores de todos los equipos de forma secuencial
    """
    todos_los_jugadores = []
    
    for i, equipo_url in enumerate(todos_equipos):
        print(f"Procesando equipo {i+1}/{len(todos_equipos)}: {equipo_url}")
        
        try:
            jugadores_equipo = get_jugadores(equipo_url)
            print(f"  - Encontrados {len(jugadores_equipo)} jugadores")
            todos_los_jugadores.extend(jugadores_equipo)
            
        except Exception as e:
            print(f"  - Error en equipo {equipo_url}: {e}")
            continue
    
    # Eliminar duplicados finales
    jugadores_unicos = list(set(todos_los_jugadores))
    print(f"\nTotal de jugadores únicos encontrados: {len(jugadores_unicos)}")
    
    return jugadores_unicos

# Ejecutar para obtener todos los jugadores
todos_jugadores = obtener_todos_jugadores_secuencial(todos_equipos)  # Limitar a los primeros 10 equipos para pruebas

Procesando equipo 1/399: https://www.sofascore.com/team/football/vfl-bochum-1848/2542
  - Encontrados 54 jugadores
Procesando equipo 2/399: https://www.sofascore.com/team/football/falkirk-fc/2363
  - Encontrados 46 jugadores
Procesando equipo 3/399: https://www.sofascore.com/team/football/dinamo-makhachkala/362016
  - Encontrados 50 jugadores
Procesando equipo 4/399: https://www.sofascore.com/team/football/polissya-zhytomyr/258536
  - Encontrados 64 jugadores
Procesando equipo 5/399: https://www.sofascore.com/team/football/fc-nordsjaelland/1292
  - Encontrados 50 jugadores
Procesando equipo 6/399: https://www.sofascore.com/team/football/udinese/2695
  - Encontrados 62 jugadores
Procesando equipo 7/399: https://www.sofascore.com/team/football/as-monaco/1653
  - Encontrados 52 jugadores
Procesando equipo 8/399: https://www.sofascore.com/team/football/espanyol/2814
  - Encontrados 56 jugadores
Procesando equipo 9/399: https://www.sofascore.com/team/football/liverpool/44
  - Encontrados 60

In [13]:
jugadores= pd.DataFrame(todos_jugadores, columns=['url'])
jugadores.to_csv('jugadores_sofascore.csv', index=False)

In [14]:
todos_jugadores = pd.read_csv('jugadores_sofascore.csv')['url'].tolist()

In [17]:
from tqdm import tqdm

In [18]:
import psutil
import time

class MonitorSimple:
    def __init__(self):
        self.inicio_tiempo = None
        self.inicio_cpu = None
        self.inicio_memoria = None
        self.max_cpu = 0
        self.max_memoria = 0
        self.muestras_cpu = []
        self.muestras_memoria = []
    
    def iniciar(self):
        """Inicia el monitoreo"""
        self.inicio_tiempo = time.time()
        self.inicio_cpu = psutil.cpu_percent()
        self.inicio_memoria = psutil.virtual_memory().percent
        print("🔍 Monitoreo de recursos iniciado...")
    
    def actualizar(self):
        """Actualiza las métricas (llamar periódicamente durante el scraping)"""
        cpu_actual = psutil.cpu_percent()
        memoria_actual = psutil.virtual_memory().percent
        
        self.max_cpu = max(self.max_cpu, cpu_actual)
        self.max_memoria = max(self.max_memoria, memoria_actual)
        self.muestras_cpu.append(cpu_actual)
        self.muestras_memoria.append(memoria_actual)
    
    def finalizar(self):
        """Muestra la métrica final"""
        tiempo_total = time.time() - self.inicio_tiempo
        cpu_promedio = sum(self.muestras_cpu) / len(self.muestras_cpu) if self.muestras_cpu else 0
        memoria_promedio = sum(self.muestras_memoria) / len(self.muestras_memoria) if self.muestras_memoria else 0
        
        print("\n" + "="*60)
        print("📊 MÉTRICAS FINALES DE RECURSOS")
        print("="*60)
        print(f"⏱️  Tiempo total: {tiempo_total:.1f} segundos")
        print(f"🖥️  CPU - Promedio: {cpu_promedio:.1f}% | Máximo: {self.max_cpu:.1f}%")
        print(f"💾 RAM - Promedio: {memoria_promedio:.1f}% | Máximo: {self.max_memoria:.1f}%")
        print(f"📈 Muestras tomadas: {len(self.muestras_cpu)}")
        print("="*60)

# Crear instancia global
monitor = MonitorSimple()

In [26]:
def obtener_todos_datos_jugadores(lista_jugadores):
    """
    Obtiene todos los datos de todos los jugadores con métrica final de recursos
    """
    # Iniciar monitoreo
    monitor.iniciar()
    
    datos_jugadores = []
    errores = []
    
    for i, jugador_url in tqdm(enumerate(lista_jugadores)):
        print(f"Procesando jugador {i+1}/{len(lista_jugadores)}: {jugador_url}")
        
        # Actualizar métricas cada 10 jugadores
        if i % 10 == 0:
            monitor.actualizar()
        
        try:
            options = Options()
            options.add_argument('--headless')
            options.add_argument('--disable-blink-features=AutomationControlled')
            options.add_experimental_option("excludeSwitches", ["enable-automation"])
            options.add_experimental_option('useAutomationExtension', False)
            
            driver = webdriver.Chrome(options=options)
            driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
            
            driver.get(jugador_url)
            wait = WebDriverWait(driver, 2)
            
            # Diccionario para almacenar datos del jugador
            datos_jugador = {'url': jugador_url}
            
            # 1. Nombre del jugador
            try:
                nombre_element = driver.find_element(By.XPATH, "//h1[contains(@class, 'Text')]")
                datos_jugador['nombre'] = nombre_element.text.strip()
            except:
                pass
            
            # 2. Posición
            try:
                posicion_element = driver.find_element(By.XPATH, "//span[contains(@class, 'Text') and contains(@class, 'fzFbgV')]")
                datos_jugador['posicion'] = posicion_element.text.strip()
            except:
                pass
            
            # 3. Equipo actual
            try:
                equipo_element = driver.find_element(By.XPATH, "//a[contains(@href, '/team/football')]")
                datos_jugador['equipo'] = equipo_element.text.strip().split("\n")[0]
            except:
                pass
            
            # 4. Nacionalidad
            try:
                nacionalidad_element = driver.find_element(By.XPATH, "//img[contains(@alt, 'country flag')]")
                datos_jugador['nacionalidad'] = nacionalidad_element.get_attribute('alt').replace(' country flag', '')
            except:
                pass
            
            # 5. Edad
            try:
                edad_element = driver.find_element(By.XPATH, "//span[contains(text(), 'years old')]")
                datos_jugador['edad'] = edad_element.text.replace(' years old', '').strip()
            except:
                pass
            
            # 6. Altura
            try:
                altura_element = driver.find_element(By.XPATH, "//span[contains(text(), 'cm')]")
                datos_jugador['altura'] = altura_element.text.strip()
            except:
                pass
            
            # 7. Valor de mercado
            try:
                valor_element = driver.find_element(By.XPATH, "//div[contains(@class, 'Text imGAlA')]")
                datos_jugador['valor_mercado'] = valor_element.text.strip()
            except:
                pass
            
            # 8. Ataque
            try:
                ataque_element = driver.find_element(By.XPATH, "//span[contains(@class, 'textStyle_table.small c_surface.s1 lh_20px d_block ta_center')]")
                datos_jugador['ataque'] = ataque_element.text.strip()
            except:
                pass
            
            # 9. Técnica
            try:
                tecnica_element = driver.find_elements(By.XPATH, "//span[contains(@class, 'textStyle_table.small c_surface.s1 lh_20px d_block ta_center')]")[1]
                datos_jugador['tecnica'] = tecnica_element.text.strip()
            except:
                pass
            
            # 10. Velocidad
            try:
                velocidad_element = driver.find_elements(By.XPATH, "//span[contains(@class, 'textStyle_table.small c_surface.s1 lh_20px d_block ta_center')]")[2]
                datos_jugador['velocidad'] = velocidad_element.text.strip()
            except:
                pass
            
            # 11. Defensa
            try:
                defensa_element = driver.find_elements(By.XPATH, "//span[contains(@class, 'textStyle_table.small c_surface.s1 lh_20px d_block ta_center')]")[3]
                datos_jugador['defensa'] = defensa_element.text.strip()
            except:
                pass
            
            # 12. Creatividad
            try:
                creatividad_element = driver.find_elements(By.XPATH, "//span[contains(@class, 'textStyle_table.small c_surface.s1 lh_20px d_block ta_center')]")[4]
                datos_jugador['creatividad'] = creatividad_element.text.strip()
            except:
                pass
            
            # 13. Liga actual
            try:
                liga_element = driver.find_element(By.XPATH, "//a[contains(@href, '/tournament/football')]")
                datos_jugador['liga'] = liga_element.text.strip()
            except:
                pass
            
            # 14. Partidos jugados
            try:
                partidos_element = driver.find_element(By.XPATH, "//span[contains(@class, 'Text fftNCK')]")
                datos_jugador['partidos'] = partidos_element.text.strip()
            except:
                pass
            
            # 15. Valoración media
            try:
                valoracion_element = driver.find_element(By.XPATH, "//span[@aria-valuenow and @role='meter']")
                datos_jugador['valoracion_media'] = valoracion_element.get_attribute('aria-valuenow')
            except:
                pass
            
            datos_jugadores.append(datos_jugador)
            print(datos_jugador.values())

        except Exception as e:
            print(f"  ❌ Error en jugador {jugador_url}: {e}")
            errores.append((jugador_url, str(e)))
            continue
        
        finally:
            if 'driver' in locals():
                driver.quit()
        # Guardar resultado final
    df_final = pd.DataFrame(datos_jugadores)
    df_final.to_csv('jugadores_datos_completos.csv', index=False)
    
    # Guardar errores
    if errores:
        df_errores = pd.DataFrame(errores)
        df_errores.to_csv('jugadores_errores.csv', index=False)
    
    # Tomar muestra final y mostrar métricas
    monitor.actualizar()
    monitor.finalizar()
    
    print(f"\n🎉 Proceso completado!")
    print(f"✅ Jugadores procesados exitosamente: {len(datos_jugadores)}")
    print(f"❌ Errores encontrados: {len(errores)}")
    
    return datos_jugadores, errores


In [ ]:
# Agregar al inicio de tu celda
import psutil
import time

datos_finales = obtener_todos_datos_jugadores(todos_jugadores[:2000])


🔍 Monitoreo de recursos iniciado...


0it [00:00, ?it/s]

Procesando jugador 1/2000: https://www.sofascore.com/football/player/dusan-stoiljkovic/547776
dict_values(['https://www.sofascore.com/football/player/dusan-stoiljkovic/547776', 'FK Napredak Kruševac', '385K €', '51', '49', '34', '38', '48', 'Mozzart Bet Superliga', '31 Appearances', '6.85'])


1it [00:08,  8.23s/it]

Procesando jugador 2/2000: https://www.sofascore.com/football/player/viktor-gyokeres/804508
dict_values(['https://www.sofascore.com/football/player/viktor-gyokeres/804508', 'Sporting', '70M €', '89', '72', '71', '34', '65', 'Liga Portugal Betclic', '33 Appearances', '7.94'])


...

Procesando jugador 16/2000: https://www.sofascore.com/football/player/borja-mayoral/604954
dict_values(['https://www.sofascore.com/football/player/borja-mayoral/604954', 'Getafe', '7.8M €', '74', '56', '45', '21', '53', 'LaLiga', '24 Appearances', '6.79'])


16it [02:16,  8.59s/it]

Procesando jugador 17/2000: https://www.sofascore.com/football/player/timon-wellenreuther/253875
dict_values(['https://www.sofascore.com/football/player/timon-wellenreuther/253875', 'Feyenoord', '6.2M €', '65', '59', '57', '76', '57', 'Eredivisie', '29 Appearances', '6.99'])


17it [02:24,  8.54s/it]

Procesando jugador 18/2000: https://www.sofascore.com/football/player/thibaud-verlinden/826172
dict_values(['https://www.sofascore.com/football/player/thibaud-verlinden/826172', 'Oud-Heverlee Leuven', '1.8M €', '64', '56', '38', '26', '58', 'Pro League', '35 Appearances', '6.93'])


In [28]:
len(todos_jugadores)/3

3825.3333333333335